In [2]:
## We first load in packages we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set_style("whitegrid")

## Read csv, check basic info and duplicates

In [3]:
df = pd.read_csv("../../Data/used_cars.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [113]:
df.duplicated().sum()


np.int64(0)

In [114]:
df.dtypes

brand           object
model           object
model_year       int64
milage          object
fuel_type       object
engine          object
transmission    object
ext_col         object
int_col         object
accident        object
clean_title     object
price           object
dtype: object

In [116]:
df.nunique()


brand             57
model           1898
model_year        34
milage          2818
fuel_type          7
engine          1146
transmission      62
ext_col          319
int_col          156
accident           2
clean_title        1
price           1569
dtype: int64

## Investigate "fuel_type" column

In [117]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [118]:
fuel_others = df.loc[(df['fuel_type']=='–')|(df['fuel_type']=='not supported')|pd.isna(df['fuel_type'])]
fuel_others.shape

(217, 12)

In [119]:
fuel_not_supp = df.loc[df['fuel_type']=='not supported']
fuel_not_supp.shape

(2, 12)

#### Show that {Fuel_type == en dash ('–') } if and only if {engine == en dash ('–')}

In [120]:
fuel_dash = df.loc[df['fuel_type']=='–']

fuel_dash.shape

(45, 12)

In [121]:
engine_dash = df.loc[df['engine']=='–']
engine_dash.shape

(45, 12)

In [122]:
fuel_engine_both_dash = df.loc[(df['engine']=='–')&(df['fuel_type']=='–')]
fuel_engine_both_dash.shape

(45, 12)

In [123]:
fuel_dash.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
130,Chrysler,Pacifica Touring,2017,"87,305 mi.",–,–,9-Speed A/T,Silver,Black,None reported,Yes,"$9,000"
257,Toyota,Land Cruiser Base,1993,"231,500 mi.",–,–,A/T,White,Gray,None reported,Yes,"$29,500"
338,Mazda,Mazda3 s Grand Touring,2016,"85,000 mi.",–,–,A/T,Black,Black,At least 1 accident or damage reported,Yes,"$18,500"
491,Chrysler,Pacifica Touring,2017,"63,000 mi.",–,–,9-Speed A/T,White,Beige,None reported,Yes,"$20,000"
692,Honda,Civic EX,1993,"58,981 mi.",–,–,A/T,White,Beige,None reported,Yes,"$9,599"


In [124]:
fuel_na = df.loc[pd.isna(df['fuel_type'])]
fuel_na.shape


(170, 12)

In [125]:
fuel_na.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
9,Tesla,Model X Long Range Plus,2020,"34,000 mi.",NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,"$69,950"
44,Lucid,Air Grand Touring,2022,"3,552 mi.",NaN,536.0HP Electric Motor Electric Fuel System,1-Speed A/T,Red,Beige,None reported,Yes,"$119,999"
68,Lucid,Air Grand Touring,2022,"4,900 mi.",NaN,536.0HP Electric Motor Electric Fuel System,1-Speed A/T,Red,Black,None reported,Yes,"$99,000"
92,Rivian,R1S Adventure Package,2023,"2,800 mi.",NaN,835.0HP Electric Motor Electric Fuel System,1-Speed A/T,White,Green,None reported,Yes,"$92,000"
122,Rivian,R1S Adventure Package,2023,"2,500 mi.",NaN,835.0HP Electric Motor Electric Fuel System,A/T,Green,White,None reported,Yes,"$94,000"


### Conclusion:
For the values in the fuel column, NaN corresponds to electric motor car (I have checked the original csv file with Excel, and sort the column fuel_type, most fuel_type==NaN rows have the key word "Electric" contained in their "engine" column, a few rows have "battery" or "dual motor" in their "engine" column. I googled those vocabularies, they are all electric motor car.), dash corresponds to missing value (there are 45 rows, and they also miss the values for engine), "not supported" (only 2 rows have this, maybe we can just drop them) corresponds to Toyota Mirai models. 

To deal with this, we can fill in the NaN values by "Electric", and drop the missing values and the two Toyota Mirai rows. (I do so as below.)
Another way to deal with the 45 missing values, is to google the model type and fill in the information by hand.  

In [ ]:
df['fuel_type'].fillna('electric', inplace=True)

In [127]:
df = df[(df.fuel_type != '–')&(df.fuel_type != 'not supported')]


In [128]:
df['fuel_type'].value_counts()


fuel_type
Gasoline          3309
Hybrid             194
electric           170
E85 Flex Fuel      139
Diesel             116
Plug-In Hybrid      34
Name: count, dtype: int64

## Clean price, milage columns and convert year column to age

Since price and milage are of string type, we convert them to float type. 

We convert year value to age (age=2024-year). For example, a 2014 car has age 2024-2014 = 10.

In [129]:
def clean_price_column(text:str) -> float:
    text = text[1:] # removes leading $
    return float(text.replace(',','')) # removes comma and converts to float

df['price'] = df['price'].apply(clean_price_column)

In [130]:
def clean_milage_column(text:str) -> float:
    text = text.replace(',','')
    return float(text.replace(' mi.', ''))

df['milage'] = df['milage'].apply(clean_milage_column)

In [131]:
def year_to_age(y:int) -> int:
    return 2024-y

df['model_year'] = df['model_year'].apply(year_to_age)
df.rename(columns = {'model_year':'age'},inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3962 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         3962 non-null   object 
 1   model         3962 non-null   object 
 2   age           3962 non-null   int64  
 3   milage        3962 non-null   float64
 4   fuel_type     3962 non-null   object 
 5   engine        3962 non-null   object 
 6   transmission  3962 non-null   object 
 7   ext_col       3962 non-null   object 
 8   int_col       3962 non-null   object 
 9   accident      3850 non-null   object 
 10  clean_title   3367 non-null   object 
 11  price         3962 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 402.4+ KB


# Create new column log_price

In [132]:
df['log_price'] = np.log10(df['price'])

# Check accident column

In [133]:
df['accident'].unique()

array(['At least 1 accident or damage reported', 'None reported', nan],
      dtype=object)

In [134]:
df['accident'].value_counts()

accident
None reported                             2875
At least 1 accident or damage reported     975
Name: count, dtype: int64

In [135]:
accident_NaN = df.loc[pd.isna(df['accident'])]
accident_NaN.shape

(112, 13)

In [ ]:
df['accident'].fillna('None reported',inplace=True)

In [137]:
df['accident'].value_counts()

accident
None reported                             2987
At least 1 accident or damage reported     975
Name: count, dtype: int64

# Check clean_title column


In [138]:
df['clean_title'].unique()

array(['Yes', nan], dtype=object)

In [139]:
clean_title_NaN = df.loc[pd.isna(df['clean_title'])]
clean_title_NaN.shape

(595, 13)

In [ ]:
df['clean_title'].fillna('no',inplace=True)

In [141]:
df.clean_title.value_counts()

clean_title
Yes    3367
no      595
Name: count, dtype: int64

# Clean color columns
 

First, let's see what are the top 20 popular colors for ext_color and int_color. The result shows that black is the most popular for both ext and int color. We can impute the missing values with black. 

In [142]:
df.ext_col.value_counts()[:20]

ext_col
Black                                 900
White                                 804
Gray                                  490
Silver                                369
Blue                                  344
Red                                   255
Green                                  70
Brown                                  43
Gold                                   41
Beige                                  38
Orange                                 36
Yellow                                 29
–                                      15
Bright White Clearcoat                 14
Diamond Black                          14
Summit White                           14
Purple                                 11
Alpine White                            8
Granite Crystal Clearcoat Metallic      7
Silver Ice Metallic                     7
Name: count, dtype: int64

In [143]:
df.int_col.value_counts()[:20]

int_col
Black                  2008
Beige                   534
Gray                    458
Brown                   156
–                       129
White                   125
Red                     125
Jet Black                59
Ebony                    44
Blue                     26
Orange                   20
Silver                   16
Global Black             13
Charcoal                 12
Nero Ade                 10
Gold                     10
Graphite                  8
Green                     7
Diesel Gray / Black       6
Black Onyx                5
Name: count, dtype: int64

Color names like 'Charcoal Black', 'Summit White' can be classified as 'black' and 'white'. Next, we build a function to extract color names. And then we apply the function to the color columns. 

In [144]:
#Import regular expression package. 
import re

In [145]:
def extract_ext_color(text):
    if text == '–': return 'black'
    color_set = {'black','white','gray','silver','blue', 'red'}
    text = text.lower()
    for color in color_set:
        if re.search(color, text):
            return color
    return 'other'

def extract_int_color(text):
    if text == '–': return 'black'
    color_set = {'black','beige', 'gray', 'brown', 'white', 'red'}
    text = text.lower()
    for color in color_set:
        if re.search(color, text):
            return color
    return 'other'


In [146]:
df['ext_col'] = df['ext_col'].apply(extract_ext_color)
df['int_col'] = df['int_col'].apply(extract_int_color)

In [147]:
df.ext_col.value_counts()

ext_col
black     1026
white      910
gray       528
silver     416
blue       390
other      387
red        305
Name: count, dtype: int64

In [148]:
df.int_col.value_counts()

int_col
black    2255
beige     552
gray      469
other     255
brown     161
red       141
white     129
Name: count, dtype: int64

# Clean transmission column

In [149]:
df['transmission'].unique()

array(['6-Speed A/T', '8-Speed Automatic', 'Automatic', '7-Speed A/T',
       'F', 'A/T', '8-Speed A/T', 'Transmission w/Dual Shift Mode',
       '9-Speed Automatic', '6-Speed M/T', '10-Speed A/T', '9-Speed A/T',
       '5-Speed A/T', '1-Speed A/T', 'Automatic CVT',
       '7-Speed Automatic with Auto-Shift', 'CVT-F', 'M/T',
       '6-Speed Automatic with Auto-Shift', '10-Speed Automatic',
       'CVT Transmission', '4-Speed A/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T',
       '8-Speed Automatic with Auto-Shift', '5-Speed Automatic',
       '8-SPEED AT', '1-Speed Automatic', '5-Speed M/T', 'Manual',
       '6-Speed Manual', 'Variable', '2', '7-Speed Manual',
       'Automatic, 9-Spd 9G-Tronic', 'Automatic, 8-Spd',
       'Automatic, 8-Spd Sport w/Sport & Manual Modes', '–',
       'Auto, 6-Spd w/CmdShft', 'Transmission Overdrive Switch',
       '7-Speed Automatic', 'Automatic, 8-Spd PDK Dual-Clutch',
       'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Ma

In [150]:
def extract_transmission_type(text):
    if text == '–': return 'black'
    auto_set = {'a/t', 'cvt','cvt-f','automatic', 'auto,', 'switch',
                'dual-clutch', 'at/mt','single-speed fixed gear',
                'transmission overdrive switch','at', 'w/dual shift mode','black'}
    manual_set = {'mamual','mt','m/t','manual,'}
    text = text.lower()
    if text == 'f' or text == 'variable' or text=='2' or text=='6-speed' or text=='7-speed' or text=='scheduled for or in production': return 'automatic'
    if text == '7-speed manual' or text == '6-speed manual' or text == '8-speed manual': return 'manual'
    for at in auto_set:
        if re.search(at, text):
            return 'automatic'
    for mt in manual_set:
        if re.search(mt, text):
            return 'manual'
    return text

In [154]:
df['transmission'] = df['transmission'].apply(extract_transmission_type)

In [155]:
df['transmission'].value_counts()

transmission
automatic    3600
manual        362
Name: count, dtype: int64

# Split the data into electric and non-electric

In [ ]:
#df_electric = df[df.fuel_type == 'electric'].copy()
#df_non_electric = df[df.fuel_type != 'electric'].copy()

In [156]:
df['electric'] = pd.get_dummies(df['fuel_type'])['electric']

In [165]:
df['electric'].value_counts()

electric
False    3792
True      170
Name: count, dtype: int64

# Drop outliers

Drop outliers for age, mileage, price columns, with upper limit = 99.5 percentile, lower limit = 0.5 percentile.

In [157]:
price_lower_limit, price_upper_limit = np.percentile(a=df.price,q=[0.5,99.5])
milage_lower_limit, milage_upper_limit = np.percentile(a=df.milage,q=[0.5,99.5])
age_lower_limit, age_upper_limit = np.percentile(a=df.age,q=[0.5,99.5])
print(price_lower_limit,price_upper_limit)
print(milage_lower_limit,milage_upper_limit)
print(age_lower_limit,age_upper_limit)

3500.0 367773.2550000065
299.805 244039.00000000003
1.0 26.0


In [158]:
df_filtered = df[(df.price>price_lower_limit) & (df.price<price_upper_limit)\
                  & (df.milage>milage_lower_limit) & (df.milage<milage_upper_limit)\
                  & (df.age>age_lower_limit) & (df.age<age_upper_limit)]

df_filtered.shape

(3645, 14)

# Train-test split stratified by electric/non-electric

In [159]:
from sklearn.model_selection import train_test_split

In [167]:
X = df_filtered.drop(['price'],axis=1)
y = df_filtered['price']

In [168]:
X_train, X_test, y_train, y_test  = train_test_split(X, y,
                                            shuffle=True,
                                            random_state=21,
                                            test_size = .2,
                                            stratify=df_filtered['electric'])

In [169]:
print("The shape of X_train is",X_train.shape)
print("The shape of X_test is",X_test.shape)
print("The length of y_train is",len(y_train))
print("The length of y_test is",len(y_test))

The shape of X_train is (2916, 13)
The shape of X_test is (729, 13)
The length of y_train is 2916
The length of y_test is 729


In [170]:
continuous_features = ['milage','age']

In [171]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [172]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits = 5,
              shuffle = True,
              random_state = 582)
kfold.split(X_train, y_train)

<generator object _BaseKFold.split at 0x000001601CE1AF00>

In [173]:
for train_index, test_index in kfold.split(X_train, y_train):
    print("Train index:", train_index)
    print("Test index:", test_index)
    print()
    print()

Train index: [   0    2    4 ... 2913 2914 2915]
Test index: [   1    3   12   18   19   31   33   40   41   42   44   47   52   59
   64   70   74   76   77   78   80   82   83   91   96  100  107  114
  121  123  129  141  142  144  156  160  167  172  176  183  184  187
  193  195  199  205  211  213  214  218  225  232  234  235  239  244
  256  260  265  274  281  299  301  311  317  321  324  338  369  370
  371  372  373  383  385  389  390  397  398  403  410  413  416  418
  426  428  431  444  445  460  464  467  472  476  477  487  491  496
  508  518  523  525  529  535  536  538  539  544  549  552  555  557
  558  568  573  577  583  588  595  607  612  613  619  625  631  633
  634  643  651  673  674  679  682  685  688  693  698  705  710  711
  720  732  745  746  747  766  767  769  771  773  780  783  785  797
  800  815  822  836  837  838  840  841  842  845  860  863  865  867
  868  872  895  908  912  915  927  929  933  934  938  940  942  946
  952  962  968 